In [1]:
import csv
import itertools
import numpy
import random
from CWHashingFamily import CWHashingFamily

SHINGLES_LEN = 4
PRIME = 997
BINS = 900

def shingles(text, shingleSize):
    return [text[i:i+shingleSize] for i in xrange(0,len(text)-(shingleSize+1))]


def minhash(shinglesList, seed):
    random.seed(seed)
    minhash = -1
    for shingle in shinglesList:
        cwFactory = CWHashingFamily(BINS, PRIME)
        numericHash = cwFactory.numericFunction(random.randrange(1,PRIME), random.randrange(0,PRIME))
        stringHash = cwFactory.stringFunction(random.randrange(0,PRIME),numericHash)
        shHash = stringHash.hash(shingle)
        if ((minhash == -1) or (shHash < minhash)):
            minhash = shHash
    return minhash

In [2]:
ID = 0
TEXT = 9
R = 4
B = 8

trainRDD = sc.textFile("train_sin_repeticiones.csv").map(lambda line: line.split(","))

trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")

trainRDD = trainRDD.map(lambda line: (line[ID], shingles(line[TEXT],SHINGLES_LEN)))

trainRDD = trainRDD.map(lambda line: (line[0], [ minhash(line[1], i) for i in xrange(0, R*B) ] ))

print trainRDD.take(10)


[(u'492346', [3, 2, 0, 9, 1, 0, 1, 3, 6, 0, 0, 5, 14, 9, 0, 0, 0, 0, 7, 1, 3, 0, 1, 0, 2, 0, 0, 0, 3, 2, 0, 3]), (u'343811', [15, 5, 6, 3, 1, 3, 3, 0, 4, 0, 1, 2, 1, 0, 4, 6, 0, 4, 4, 1, 0, 5, 1, 1, 5, 5, 0, 3, 0, 5, 1, 2]), (u'347257', [20, 1, 3, 19, 24, 11, 8, 0, 33, 10, 2, 3, 21, 19, 10, 8, 2, 10, 12, 24, 0, 5, 24, 25, 1, 5, 6, 28, 0, 1, 3, 9]), (u'225529', [25, 29, 67, 38, 64, 6, 0, 42, 85, 39, 37, 2, 14, 18, 2, 73, 82, 2, 0, 125, 240, 21, 39, 3, 23, 2, 17, 68, 32, 29, 45, 20]), (u'485488', [1, 1, 12, 0, 0, 7, 3, 0, 5, 0, 1, 4, 0, 0, 5, 0, 0, 5, 5, 0, 1, 3, 0, 2, 1, 3, 5, 5, 0, 1, 3, 3]), (u'74981', [4, 2, 1, 2, 2, 1, 0, 10, 2, 0, 2, 1, 1, 2, 6, 4, 0, 6, 2, 2, 13, 7, 2, 0, 6, 0, 2, 2, 10, 6, 2, 0]), (u'204180', [6, 1, 4, 0, 3, 3, 3, 5, 0, 1, 8, 0, 0, 0, 0, 1, 2, 0, 0, 6, 5, 6, 6, 6, 1, 9, 1, 1, 5, 1, 8, 3]), (u'454372', [3, 5, 3, 1, 2, 1, 6, 0, 0, 4, 1, 2, 0, 1, 2, 2, 0, 2, 2, 2, 3, 3, 2, 0, 5, 5, 4, 4, 0, 5, 1, 3]), (u'112235', [3, 2, 7, 7, 3, 4, 15, 6, 0, 2, 29, 5, 14, 6, 24, 4, 

In [3]:
from LSH import LSH
from Record import Record

lsh = LSH(R,B)

records = trainRDD.flatMap(lambda line: [lsh.add(Record(line))])


In [4]:
text = "for about 5 years, especially during the winter months, i used e down with several very painful sore throats, many of which would escalate into strep throat or a sinus infection.  i haven't had one since i started drinking kukicha tea 2 years ago.  it's isn't carried in many stores so i'm glad i can get it through amazon now and that it's eligible for free shipping.i'm not a doctor and this isn't medical advice, but if you are having a problem with sore throats this is worth a shot."
test = (459662,shingles(text,4))
test = (test[0], [ minhash(test[1], i) for i in xrange(0, R*B) ] )
testRecord = Record(test)

In [5]:
print lsh.getAllSimilarRecords(testRecord)

[]
